In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import isnan, when, count, col, translate
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql import functions as F

sqlContext = SQLContext(sc)

In [2]:
df_spark = sqlContext.read.csv("FileStore/tables/googleplaystore.csv", header=True, inferSchema = True)
df_spark.count()

Out[32]: 10841

In [3]:
df_spark.printSchema()

root
-- App: string (nullable = true)
-- Category: string (nullable = true)
-- Rating: string (nullable = true)
-- Reviews: string (nullable = true)
-- Size: string (nullable = true)
-- Installs: string (nullable = true)
-- Type: string (nullable = true)
-- Price: string (nullable = true)
-- Content Rating: string (nullable = true)
-- Genres: string (nullable = true)
-- Last Updated: string (nullable = true)
-- Current Ver: string (nullable = true)
-- Android Ver: string (nullable = true)

In [4]:
df_spark.show(5)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
 App| Category|Rating|Reviews|Size| Installs|Type|Price|Content Rating| Genres| Last Updated| Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
Photo Editor & Ca...|ART_AND_DESIGN| 4.1| 159| 19M| 10,000+|Free| 0| Everyone| Art & Design| January 7, 2018| 1.0.0|4.0.3 and up|
 Coloring book moana|ART_AND_DESIGN| 3.9| 967| 14M| 500,000+|Free| 0| Everyone|Art & Design;Pret...|January 15, 2018| 2.0.0|4.0.3 and up|
U Launcher Lite –...|ART_AND_DESIGN| 4.7| 87510|8.7M| 5,000,000+|Free| 0| Everyone| Art & Design| August 1, 2018| 1.2.4|4.0.3 and up|
Sketch - Draw & P...|ART_AND_DESIGN| 4.5| 215644| 25M|50,000,000+|Free| 0| Teen| Art & Design| June 8, 2018|Varies with device| 4.2 and up|
Pixel Draw - Numb...|ART_AND_DESIGN| 4.3| 967|2.8M| 100,000+|Free| 0| Everyone|Art & Design;Crea...| June 20, 2018| 1.1| 4.4 and up|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
only showing top 5 rows

In [5]:
df_spark.select("App").show(5)

+--------------------+
 App|
+--------------------+
Photo Editor & Ca...|
 Coloring book moana|
U Launcher Lite –...|
Sketch - Draw & P...|
Pixel Draw - Numb...|
+--------------------+
only showing top 5 rows

In [6]:
df_spark.select("App", "Category", "Rating").show(5)

+--------------------+--------------+------+
 App| Category|Rating|
+--------------------+--------------+------+
Photo Editor & Ca...|ART_AND_DESIGN| 4.1|
 Coloring book moana|ART_AND_DESIGN| 3.9|
U Launcher Lite –...|ART_AND_DESIGN| 4.7|
Sketch - Draw & P...|ART_AND_DESIGN| 4.5|
Pixel Draw - Numb...|ART_AND_DESIGN| 4.3|
+--------------------+--------------+------+
only showing top 5 rows

In [7]:
df_spark.select([count(when(col(c).isNull(), c)).alias(c) for c in df_spark.columns]).show()

+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
App|Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 1| 1|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+

In [8]:
df_spark = df_spark.dropna(how='any')

In [9]:
df_spark = df_spark.withColumn('Size', translate('Size', 'Mk', ''))
df_spark = df_spark.withColumn('Installs', translate('Installs', '+,', ''))
df_spark = df_spark.withColumn('Price', translate('Price', '$', ''))

df_spark.show(5)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+--------------------+----------------+------------------+------------+
 App| Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating| Genres| Last Updated| Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+--------------------+----------------+------------------+------------+
Photo Editor & Ca...|ART_AND_DESIGN| 4.1| 159| 19| 10000|Free| 0| Everyone| Art & Design| January 7, 2018| 1.0.0|4.0.3 and up|
 Coloring book moana|ART_AND_DESIGN| 3.9| 967| 14| 500000|Free| 0| Everyone|Art & Design;Pret...|January 15, 2018| 2.0.0|4.0.3 and up|
U Launcher Lite –...|ART_AND_DESIGN| 4.7| 87510| 8.7| 5000000|Free| 0| Everyone| Art & Design| August 1, 2018| 1.2.4|4.0.3 and up|
Sketch - Draw & P...|ART_AND_DESIGN| 4.5| 215644| 25|50000000|Free| 0| Teen| Art & Design| June 8, 2018|Varies with device| 4.2 and up|
Pixel Draw - Numb...|ART_AND_DESIGN| 4.3| 967| 2.8| 100000|Free| 0| Everyone|Art & Design;Crea...| June 20, 2018| 1.1| 4.4 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+--------------------+----------------+------------------+------------+
only showing top 5 rows

In [10]:
df_spark = df_spark.where((col("Size") != "Varies with device" ))

In [11]:
df_spark = df_spark.withColumn("Installs", df_spark["Installs"].cast(IntegerType()))
df_spark = df_spark.withColumn("Rating", df_spark["Rating"].cast(FloatType()))
df_spark = df_spark.withColumn("Size", df_spark["Size"].cast(FloatType()))
df_spark = df_spark.withColumn("Price", df_spark["Price"].cast(FloatType()))

df_spark.printSchema()

root
-- App: string (nullable = true)
-- Category: string (nullable = true)
-- Rating: float (nullable = true)
-- Reviews: string (nullable = true)
-- Size: float (nullable = true)
-- Installs: integer (nullable = true)
-- Type: string (nullable = true)
-- Price: float (nullable = true)
-- Content Rating: string (nullable = true)
-- Genres: string (nullable = true)
-- Last Updated: string (nullable = true)
-- Current Ver: string (nullable = true)
-- Android Ver: string (nullable = true)

In [12]:
df_spark = df_spark.withColumnRenamed('Size', 'Size(M)')
df_spark = df_spark.withColumnRenamed('Installs', 'Installs(+)')
df_spark = df_spark.withColumnRenamed('Price', 'Price($)')
df_spark.show(5)

+--------------------+--------------+------+-------+-------+-----------+----+--------+--------------+--------------------+----------------+------------------+------------+
 App| Category|Rating|Reviews|Size(M)|Installs(+)|Type|Price($)|Content Rating| Genres| Last Updated| Current Ver| Android Ver|
+--------------------+--------------+------+-------+-------+-----------+----+--------+--------------+--------------------+----------------+------------------+------------+
Photo Editor & Ca...|ART_AND_DESIGN| 4.1| 159| 19.0| 10000|Free| 0.0| Everyone| Art & Design| January 7, 2018| 1.0.0|4.0.3 and up|
 Coloring book moana|ART_AND_DESIGN| 3.9| 967| 14.0| 500000|Free| 0.0| Everyone|Art & Design;Pret...|January 15, 2018| 2.0.0|4.0.3 and up|
U Launcher Lite –...|ART_AND_DESIGN| 4.7| 87510| 8.7| 5000000|Free| 0.0| Everyone| Art & Design| August 1, 2018| 1.2.4|4.0.3 and up|
Sketch - Draw & P...|ART_AND_DESIGN| 4.5| 215644| 25.0| 50000000|Free| 0.0| Teen| Art & Design| June 8, 2018|Varies with device| 4.2 and up|
Pixel Draw - Numb...|ART_AND_DESIGN| 4.3| 967| 2.8| 100000|Free| 0.0| Everyone|Art & Design;Crea...| June 20, 2018| 1.1| 4.4 and up|
+--------------------+--------------+------+-------+-------+-----------+----+--------+--------------+--------------------+----------------+------------------+------------+
only showing top 5 rows

In [13]:
df_spark = df_spark.drop("Reviews", "Genres", "Current Ver")
df_spark.show(5)

+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
 App| Category|Rating|Size(M)|Installs(+)|Type|Price($)|Content Rating| Last Updated| Android Ver|
+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
Photo Editor & Ca...|ART_AND_DESIGN| 4.1| 19.0| 10000|Free| 0.0| Everyone| January 7, 2018|4.0.3 and up|
 Coloring book moana|ART_AND_DESIGN| 3.9| 14.0| 500000|Free| 0.0| Everyone|January 15, 2018|4.0.3 and up|
U Launcher Lite –...|ART_AND_DESIGN| 4.7| 8.7| 5000000|Free| 0.0| Everyone| August 1, 2018|4.0.3 and up|
Sketch - Draw & P...|ART_AND_DESIGN| 4.5| 25.0| 50000000|Free| 0.0| Teen| June 8, 2018| 4.2 and up|
Pixel Draw - Numb...|ART_AND_DESIGN| 4.3| 2.8| 100000|Free| 0.0| Everyone| June 20, 2018| 4.4 and up|
+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
only showing top 5 rows

In [14]:
df_spark.filter(df_spark["Content Rating"] == "Teen").show(10)

+--------------------+-------------------+------+-------+-----------+----+--------+--------------+--------------+------------+
 App| Category|Rating|Size(M)|Installs(+)|Type|Price($)|Content Rating| Last Updated| Android Ver|
+--------------------+-------------------+------+-------+-----------+----+--------+--------------+--------------+------------+
Sketch - Draw & P...| ART_AND_DESIGN| 4.5| 25.0| 50000000|Free| 0.0| Teen| June 8, 2018| 4.2 and up|
Tattoo Name On My...| ART_AND_DESIGN| 4.2| 20.0| 10000000|Free| 0.0| Teen| April 2, 2018| 4.1 and up|
 I Creative Idea| ART_AND_DESIGN| 4.7| 4.2| 10000|Free| 0.0| Teen|April 27, 2018| 4.1 and up|
Android Auto - Ma...| AUTO_AND_VEHICLES| 4.2| 16.0| 10000000|Free| 0.0| Teen| July 11, 2018| 5.0 and up|
Selfie Camera Pho...| BEAUTY| 4.1| 30.0| 50000|Free| 0.0| Teen| July 24, 2018|4.0.3 and up|
Skin Care and Nat...| BEAUTY| NaN| 7.4| 100000|Free| 0.0| Teen| July 17, 2018| 4.1 and up|
 Book store|BOOKS_AND_REFERENCE| 4.4| 5.4| 1000000|Free| 0.0| Teen| June 6, 2018| 4.0 and up|
Free Books - Spir...|BOOKS_AND_REFERENCE| 4.8| 5.0| 1000000|Free| 0.0| Teen|August 1, 2018|4.0.3 and up|
 Litnet - E-books|BOOKS_AND_REFERENCE| 4.6| 15.0| 100000|Free| 0.0| Teen| July 4, 2018| 4.0 and up|
GANMA! - All orig...| COMICS| 4.7| 27.0| 1000000|Free| 0.0| Teen| July 23, 2018| 4.1 and up|
+--------------------+-------------------+------+-------+-----------+----+--------+--------------+--------------+------------+
only showing top 10 rows

In [15]:
df_spark.filter(df_spark["Rating"] > 4.2).show(10)

+--------------------+--------------+------+-------+-----------+----+--------+--------------+------------------+------------+
 App| Category|Rating|Size(M)|Installs(+)|Type|Price($)|Content Rating| Last Updated| Android Ver|
+--------------------+--------------+------+-------+-----------+----+--------+--------------+------------------+------------+
U Launcher Lite –...|ART_AND_DESIGN| 4.7| 8.7| 5000000|Free| 0.0| Everyone| August 1, 2018|4.0.3 and up|
Sketch - Draw & P...|ART_AND_DESIGN| 4.5| 25.0| 50000000|Free| 0.0| Teen| June 8, 2018| 4.2 and up|
Pixel Draw - Numb...|ART_AND_DESIGN| 4.3| 2.8| 100000|Free| 0.0| Everyone| June 20, 2018| 4.4 and up|
Paper flowers ins...|ART_AND_DESIGN| 4.4| 5.6| 50000|Free| 0.0| Everyone| March 26, 2017| 2.3 and up|
Garden Coloring Book|ART_AND_DESIGN| 4.4| 33.0| 1000000|Free| 0.0| Everyone|September 20, 2017| 3.0 and up|
Kids Paint Free -...|ART_AND_DESIGN| 4.7| 3.1| 10000|Free| 0.0| Everyone| July 3, 2018|4.0.3 and up|
Text on Photo - F...|ART_AND_DESIGN| 4.4| 28.0| 1000000|Free| 0.0| Everyone| October 27, 2017| 4.1 and up|
Name Art Photo Ed...|ART_AND_DESIGN| 4.4| 12.0| 1000000|Free| 0.0| Everyone| July 31, 2018| 4.0 and up|
Mandala Coloring ...|ART_AND_DESIGN| 4.6| 21.0| 100000|Free| 0.0| Everyone| June 26, 2018| 4.4 and up|
3D Color Pixel by...|ART_AND_DESIGN| 4.4| 37.0| 100000|Free| 0.0| Everyone| August 3, 2018| 2.3 and up|
+--------------------+--------------+------+-------+-----------+----+--------+--------------+------------------+------------+
only showing top 10 rows

In [16]:
df_spark.filter(df_spark["Price($)"].between(1, 3)).show(10)

+--------------------+-------------+------+-------+-----------+----+--------+--------------+-----------------+-----------+
 App| Category|Rating|Size(M)|Installs(+)|Type|Price($)|Content Rating| Last Updated|Android Ver|
+--------------------+-------------+------+-------+-----------+----+--------+--------------+-----------------+-----------+
 Truth or Dare Pro| DATING| NaN| 20.0| 50|Paid| 1.49| Teen|September 1, 2017| 4.0 and up|
Private Dating, H...| DATING| NaN| 18.0| 100|Paid| 2.99| Everyone| July 25, 2017| 4.0 and up|
 Meme Generator|ENTERTAINMENT| 4.6| 53.0| 100000|Paid| 2.99| Mature 17+| August 3, 2018| 4.1 and up|
 The Game of Life| GAME| 4.4| 63.0| 100000|Paid| 2.99| Everyone| July 4, 2018| 4.4 and up|
 Clue| GAME| 4.6| 35.0| 100000|Paid| 1.99| Everyone 10+| July 30, 2018| 5.0 and up|
My Little Princes...| FAMILY| 4.3| 60.0| 5000|Paid| 2.99| Everyone| May 25, 2018| 4.1 and up|
 Pinkalicious Party| FAMILY| 3.0| 82.0| 500|Paid| 2.99| Everyone| June 25, 2018| 4.1 and up|
 Alizay, pirate girl| FAMILY| 4.5| 11.0| 1000|Paid| 2.99| Everyone| March 23, 2017| 2.3 and up|
 Hactar Go| FAMILY| 4.8| 3.5| 1000|Paid| 2.99| Everyone| March 30, 2018| 4.0 and up|
Whoowasit? - Best...| FAMILY| 4.0| 26.0| 5000|Paid| 2.99| Everyone| July 24, 2018| 4.1 and up|
+--------------------+-------------+------+-------+-----------+----+--------+--------------+-----------------+-----------+
only showing top 10 rows

In [17]:
df_spark.filter(df_spark["Android Ver"].startswith("4")).show(10)

+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
 App| Category|Rating|Size(M)|Installs(+)|Type|Price($)|Content Rating| Last Updated| Android Ver|
+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
Photo Editor & Ca...|ART_AND_DESIGN| 4.1| 19.0| 10000|Free| 0.0| Everyone| January 7, 2018|4.0.3 and up|
 Coloring book moana|ART_AND_DESIGN| 3.9| 14.0| 500000|Free| 0.0| Everyone|January 15, 2018|4.0.3 and up|
U Launcher Lite –...|ART_AND_DESIGN| 4.7| 8.7| 5000000|Free| 0.0| Everyone| August 1, 2018|4.0.3 and up|
Sketch - Draw & P...|ART_AND_DESIGN| 4.5| 25.0| 50000000|Free| 0.0| Teen| June 8, 2018| 4.2 and up|
Pixel Draw - Numb...|ART_AND_DESIGN| 4.3| 2.8| 100000|Free| 0.0| Everyone| June 20, 2018| 4.4 and up|
Smoke Effect Phot...|ART_AND_DESIGN| 3.8| 19.0| 50000|Free| 0.0| Everyone| April 26, 2018|4.0.3 and up|
 Infinite Painter|ART_AND_DESIGN| 4.1| 29.0| 1000000|Free| 0.0| Everyone| June 14, 2018| 4.2 and up|
Kids Paint Free -...|ART_AND_DESIGN| 4.7| 3.1| 10000|Free| 0.0| Everyone| July 3, 2018|4.0.3 and up|
Text on Photo - F...|ART_AND_DESIGN| 4.4| 28.0| 1000000|Free| 0.0| Everyone|October 27, 2017| 4.1 and up|
Name Art Photo Ed...|ART_AND_DESIGN| 4.4| 12.0| 1000000|Free| 0.0| Everyone| July 31, 2018| 4.0 and up|
+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
only showing top 10 rows

In [18]:
df_spark.filter(df_spark["Last Updated"].endswith("2018")).show(10)

+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
 App| Category|Rating|Size(M)|Installs(+)|Type|Price($)|Content Rating| Last Updated| Android Ver|
+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
Photo Editor & Ca...|ART_AND_DESIGN| 4.1| 19.0| 10000|Free| 0.0| Everyone| January 7, 2018|4.0.3 and up|
 Coloring book moana|ART_AND_DESIGN| 3.9| 14.0| 500000|Free| 0.0| Everyone|January 15, 2018|4.0.3 and up|
U Launcher Lite –...|ART_AND_DESIGN| 4.7| 8.7| 5000000|Free| 0.0| Everyone| August 1, 2018|4.0.3 and up|
Sketch - Draw & P...|ART_AND_DESIGN| 4.5| 25.0| 50000000|Free| 0.0| Teen| June 8, 2018| 4.2 and up|
Pixel Draw - Numb...|ART_AND_DESIGN| 4.3| 2.8| 100000|Free| 0.0| Everyone| June 20, 2018| 4.4 and up|
Smoke Effect Phot...|ART_AND_DESIGN| 3.8| 19.0| 50000|Free| 0.0| Everyone| April 26, 2018|4.0.3 and up|
 Infinite Painter|ART_AND_DESIGN| 4.1| 29.0| 1000000|Free| 0.0| Everyone| June 14, 2018| 4.2 and up|
Kids Paint Free -...|ART_AND_DESIGN| 4.7| 3.1| 10000|Free| 0.0| Everyone| July 3, 2018|4.0.3 and up|
Name Art Photo Ed...|ART_AND_DESIGN| 4.4| 12.0| 1000000|Free| 0.0| Everyone| July 31, 2018| 4.0 and up|
Tattoo Name On My...|ART_AND_DESIGN| 4.2| 20.0| 10000000|Free| 0.0| Teen| April 2, 2018| 4.1 and up|
+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
only showing top 10 rows

In [19]:
df_spark.filter(df_spark["App"].contains("photo")).show(10)

+--------------------+-----------+------+-------+-----------+----+--------+--------------+---------------+------------+
 App| Category|Rating|Size(M)|Installs(+)|Type|Price($)|Content Rating| Last Updated| Android Ver|
+--------------------+-----------+------+-------+-----------+----+--------+--------------+---------------+------------+
BestCam Selfie-se...| BEAUTY| 3.9| 21.0| 500000|Free| 0.0| Everyone| July 12, 2018|4.0.3 and up|
i miss you quotes...| DATING| NaN| 5.0| 100|Free| 0.0| Everyone| July 22, 2018| 4.0 and up|
Picktrainer: Indi...| EVENTS| 4.5| 9.0| 100000|Free| 0.0| Everyone| May 31, 2018| 4.1 and up|
LALALAB prints yo...|PHOTOGRAPHY| 4.7| 82.0| 1000000|Free| 0.0| Everyone| August 2, 2018| 4.1 and up|
Sweet Snap Lite -...|PHOTOGRAPHY| 4.5| 22.0| 500000|Free| 0.0| Everyone| August 3, 2018|4.0.3 and up|
Sweet Selfie - se...|PHOTOGRAPHY| 4.6| 25.0| 100000000|Free| 0.0| Everyone| August 3, 2018|4.0.3 and up|
Candy selfie - ph...|PHOTOGRAPHY| 4.4| 26.0| 10000000|Free| 0.0| Everyone| August 6, 2018|4.0.3 and up|
Muzy - Share phot...|PHOTOGRAPHY| 4.4| 13.0| 5000000|Free| 0.0| Teen|August 11, 2015| 4.0 and up|
Muzy - Share phot...|PHOTOGRAPHY| 4.4| 13.0| 5000000|Free| 0.0| Teen|August 11, 2015| 4.0 and up|
Pixgram- video ph...|PHOTOGRAPHY| 4.2| 9.2| 5000000|Free| 0.0| Everyone| July 8, 2018| 4.3 and up|
+--------------------+-----------+------+-------+-----------+----+--------+--------------+---------------+------------+
only showing top 10 rows

In [20]:
df_spark.filter((col("Last Updated").endswith("2018")) & (col("Category") == "GAME") & col("Android Ver").startswith("5")).show(10)

+--------------------+--------+------+-------+-----------+----+--------+--------------+--------------+-----------+
 App|Category|Rating|Size(M)|Installs(+)|Type|Price($)|Content Rating| Last Updated|Android Ver|
+--------------------+--------+------+-------+-----------+----+--------+--------------+--------------+-----------+
 Paint Hit| GAME| 4.4| 33.0| 10000000|Free| 0.0| Everyone| July 6, 2018| 5.0 and up|
 Tomb of the Mask| GAME| 4.1| 39.0| 5000000|Free| 0.0| Everyone| July 24, 2018| 5.0 and up|
 Tank Stars| GAME| 4.3| 98.0| 10000000|Free| 0.0| Everyone 10+| July 11, 2018| 5.0 and up|
Flip the Gun - Si...| GAME| 4.2| 86.0| 10000000|Free| 0.0| Everyone| July 26, 2018| 5.0 and up|
MMX Hill Dash 2 –...| GAME| 4.4| 79.0| 1000000|Free| 0.0| Everyone| July 19, 2018| 5.0 and up|
Partymasters - Fu...| GAME| 4.6| 63.0| 10000000|Free| 0.0| Teen| July 12, 2018| 5.0 and up|
 Clue| GAME| 4.6| 35.0| 100000|Paid| 1.99| Everyone 10+| July 30, 2018| 5.0 and up|
 Crossy Road| GAME| 4.5| 60.0| 100000000|Free| 0.0| Everyone|August 1, 2018| 5.0 and up|
 TEKKEN™| GAME| 4.2| 38.0| 5000000|Free| 0.0| Teen| July 26, 2018| 5.0 and up|
 Crossy Road| GAME| 4.5| 60.0| 100000000|Free| 0.0| Everyone|August 1, 2018| 5.0 and up|
+--------------------+--------+------+-------+-----------+----+--------+--------------+--------------+-----------+
only showing top 10 rows

In [21]:
df_spark.describe().show()

+-------+--------------------+-------------+------+-----------------+-------------------+-----------------+------------------+--------------+-----------------+------------------+
summary| App| Category|Rating| Size(M)| Installs(+)| Type| Price($)|Content Rating| Last Updated| Android Ver|
+-------+--------------------+-------------+------+-----------------+-------------------+-----------------+------------------+--------------+-----------------+------------------+
 count| 9145| 9145| 9143| 9143| 9145| 9145| 9143| 9145| 9145| 9145|
 mean| null| null| NaN|37.04179153405354| 7114623.671077091| 52378.5|1.1846253751704485| null| 0.0| NaN|
 stddev| null| null| NaN|98.88364050883112|4.619359764309197E7|70526.12324876507|17.357642974947026| null| 0.0| NaN|
 min|"""i DT"" Fútbol....| traffic jams| 1.0| 1.0| 0| 102248| 0.0| 1,000,000+| 0| 1.0 and up|
 max|🔥 Football Wallp...| WEATHER| NaN| 1020.0| 1000000000| Paid| 400.0| Unrated|September 9, 2017|Varies with device|
+-------+--------------------+-------------+------+-----------------+-------------------+-----------------+------------------+--------------+-----------------+------------------+

In [22]:
df_spark.sort("Price($)", ascending=False).show(10)

+--------------------+---------+------+-------+-----------+----+--------+--------------+-----------------+------------+
 App| Category|Rating|Size(M)|Installs(+)|Type|Price($)|Content Rating| Last Updated| Android Ver|
+--------------------+---------+------+-------+-----------+----+--------+--------------+-----------------+------------+
I'm Rich - Trump ...|LIFESTYLE| 3.6| 7.3| 10000|Paid| 400.0| Everyone| May 3, 2018| 4.1 and up|
 I am Rich| FINANCE| 4.3| 3.8| 5000|Paid| 399.99| Everyone| March 22, 2018| 4.2 and up|
 I am Rich!| FINANCE| 3.8| 22.0| 1000|Paid| 399.99| Everyone|December 11, 2017| 4.1 and up|
most expensive ap...| FAMILY| 4.3| 1.5| 100|Paid| 399.99| Everyone| July 16, 2018| 7.0 and up|
 I Am Rich Pro| FAMILY| 4.4| 2.7| 5000|Paid| 399.99| Everyone| May 30, 2017| 1.6 and up|
 💎 I'm rich|LIFESTYLE| 3.8| 26.0| 10000|Paid| 399.99| Everyone| March 11, 2018| 4.4 and up|
 I am rich(premium)| FINANCE| 3.5| 965.0| 5000|Paid| 399.99| Everyone| May 1, 2017| 4.4 and up|
 I am rich|LIFESTYLE| 3.8| 1.8| 100000|Paid| 399.99| Everyone| January 12, 2018|4.0.3 and up|
 I Am Rich Premium| FINANCE| 4.1| 4.7| 50000|Paid| 399.99| Everyone|November 12, 2017| 4.0 and up|
I am rich (Most e...| FINANCE| 4.1| 2.7| 1000|Paid| 399.99| Teen| December 6, 2017|4.0.3 and up|
+--------------------+---------+------+-------+-----------+----+--------+--------------+-----------------+------------+
only showing top 10 rows

In [23]:
df_cat_sum = df_spark.groupby("Category").agg({'Installs(+)': "sum"})
display(df_cat_sum)

Category,sum(Installs(+))
EVENTS,10672151
COMICS,17586150
SPORTS,1138949498
WEATHER,139300020
VIDEO_PLAYERS,891867720
AUTO_AND_VEHICLES,43770211
PARENTING,23971010
ENTERTAINMENT,950960000
PERSONALIZATION,998179772
HEALTH_AND_FITNESS,867411912


In [24]:
df_size_sum = df_spark.groupby("Size(M)").agg({'Installs(+)': "sum"})
display(df_size_sum)

Size(M),sum(Installs(+))
9.1,88857010
714.0,10000
9.4,12065820
364.0,1100000
18.0,237365736
6.9,39841070
64.0,38706201
82.0,301321500
308.0,5000
47.0,443638030


In [25]:
df_android_count = df_spark.groupby("Android Ver").count().sort(col("count").desc())
display(df_android_count)

Android Ver,count
4.1 and up,2316
4.0.3 and up,1453
4.0 and up,1351
4.4 and up,910
2.3 and up,635
5.0 and up,556
4.2 and up,372
2.3.3 and up,276
2.2 and up,241
3.0 and up,241


In [26]:
df_app_paid = df_spark.filter(df_spark["Type"] == "Paid")
display(df_app_paid)

App,Category,Rating,Size(M),Installs(+),Type,Price($),Content Rating,Last Updated,Android Ver
TurboScan: scan documents and receipts in PDF,BUSINESS,4.7,6.8,100000,Paid,4.99,Everyone,"March 25, 2018",4.0 and up
Tiny Scanner Pro: PDF Doc Scan,BUSINESS,4.8,39.0,100000,Paid,4.99,Everyone,"April 11, 2017",3.0 and up
TurboScan: scan documents and receipts in PDF,BUSINESS,4.7,6.8,100000,Paid,4.99,Everyone,"March 25, 2018",4.0 and up
Tiny Scanner Pro: PDF Doc Scan,BUSINESS,4.8,39.0,100000,Paid,4.99,Everyone,"April 11, 2017",3.0 and up
Calculator,DATING,2.6,6.2,1000,Paid,6.99,Everyone,"October 25, 2017",4.0 and up
Truth or Dare Pro,DATING,NaN,20.0,50,Paid,1.49,Teen,"September 1, 2017",4.0 and up
"Private Dating, Hide App- Blue for PrivacyHider",DATING,NaN,18.0,100,Paid,2.99,Everyone,"July 25, 2017",4.0 and up
Ad Blocker for SayHi,DATING,NaN,1.2,100,Paid,3.99,Teen,"August 2, 2018",4.0.3 and up
AMBW Dating App: Asian Men Black Women Interracial,DATING,3.5,17.0,100,Paid,7.99,Mature 17+,"January 21, 2017",4.0 and up
Sago Mini Hat Maker,EDUCATION,4.9,63.0,1000,Paid,3.99,Everyone,"July 24, 2017",4.0.3 and up


In [27]:
df_app_paid = df_app_paid.filter((col("Category") == "GAME") & (col("Rating") <= 5)).sort(col("Rating").desc())
display(df_app_paid)

App,Category,Rating,Size(M),Installs(+),Type,Price($),Content Rating,Last Updated,Android Ver
211:CK,GAME,5.0,38.0,10,Paid,0.99,Teen,"April 11, 2018",4.1 and up
Santa's Monster Shootout DX,GAME,5.0,33.0,50,Paid,1.99,Teen,"August 15, 2013",2.2 and up
Mu.F.O.,GAME,5.0,16.0,1,Paid,0.99,Everyone,"March 3, 2017",2.3 and up
Ra Ga Ba,GAME,5.0,20.0,1,Paid,1.49,Everyone,"February 8, 2017",2.3 and up
The Room: Old Sins,GAME,4.9,48.0,100000,Paid,4.99,Everyone,"April 18, 2018",4.4 and up
BW-Go,GAME,4.8,1.3,1000,Paid,3.49,Everyone,"June 18, 2015",2.3 and up
Cytus II,GAME,4.7,81.0,100000,Paid,1.99,Everyone 10+,"August 3, 2018",4.4 and up
An Elite Warrior Ex,GAME,4.7,93.0,100,Paid,0.99,Everyone,"November 21, 2016",4.0.3 and up
Five Nights at Freddy's 3,GAME,4.7,50.0,100000,Paid,2.99,Teen,"April 2, 2015",2.3 and up
Retro City Rampage DX,GAME,4.7,16.0,10000,Paid,2.99,Teen,"January 12, 2017",4.0 and up
